In [ ]:
#this code makes a figure that identifies which centroids from which clustering and docking methods are ranking ligands correctly/incorrectly in the top/bottom quartile

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import glob
import os

In [ ]:
#saving out OE csvs

path = '/home/jegan/bccgc4/OE_new_docking/'
home = '/home/jegan/OE_Glide_correlation/'
methods = ['TICA','TICA_CBA','PCA','PCA_CBA','GROMOS','GROMOS_CBA']
dirs = ['TICA_bkbnpos','TICA_recepatoms','PCA_nonstrided','PCA_recepatoms','GROMOS_str100','GROMOS_recepatoms']

for x in glob.glob(path+'*'):
    for v,y in zip(methods,dirs):
        if (x.rsplit('/')[5]) == (y+'_docking'):
            lines_per_file = 459
            smallfile = None
            with open('/home/jegan/bccgc4/OE_new_docking/'+y+'_docking/Docking_Results/score.txt') as bigfile:
                for lineno, line in enumerate(bigfile):
                    if lineno % lines_per_file == 0:
                        if smallfile:
                            smallfile.close()
                        cen = line[22]
                        small_filename = '/home/jegan/OE_Glide_correlation/csvs_unordered/OE_'+v+'_'+str(cen)+'.csv'
                        smallfile = open(small_filename, "w")
                    line = line.rsplit(' ')
                    newline = line[1]+','+line[2]
                    smallfile.write(newline)
                if smallfile:
                    smallfile.close()
                    
            for a in os.listdir(home+'csvs_unordered/'):
                if v in a:
                    with open(home+'csvs_unordered/'+a,'r') as centroid:
                        scores = []
                        for line in centroid:
                            lig = line.rsplit(',')
                            scores.append(lig)
                        ordered = sorted(scores, key = lambda u:float(u[1]))
                        ranked = enumerate(ordered,1)

                        with open(home+'/csvs/'+a,'w') as newfile:
                            for w,r in ranked:
                                final = r[0]+','+str(w)+','+r[1]
                                newfile.write(final)  

In [ ]:
#ordering glide csvs that were procured through glide_csvs.py

home = '/home/jegan/OE_Glide_correlation/'
methods = ['TICA','TICA_CBA','PCA','PCA_CBA','GROMOS','GROMOS_CBA']

for q in os.listdir(home+'csvs_unordered/'):
    if 'GD' in q:
        with open(home+'csvs/'+q, 'w') as newfile:
            with open(home+'csvs_unordered/'+q, 'r') as scores:

                scores = scores.readlines()

                ls = []
                for line in scores:
                    obj = line.split(',')
                    ls.append(obj)

                count = 1
                for k in ls[2:]:
                    newfile.write(k[0]+','+str(count)+','+k[1])
                    count = count+1

In [ ]:
#assigning values to the quartiles

ranks_top = [] #list of top quartile of real ligands (from experimental IC50 results)
ranks_bot = []
with open('/home/jegan/bccgc4/Answers.csv','r') as ansfile:
    i = 0
    for l in ansfile:
        item = l.split(',')
        i += 1
        if i <= 115:
            ranks_top.append(item[0]) 
        elif i >= 344:
            ranks_bot.append(item[0])
            
names = ['TICA','TICA_CBA','PCA','PCA_CBA','GROMOS','GROMOS_CBA']        
docks = ['OE','GD']
array = [] #a list of lists, where each list will be the values for ligands in numerical order for one of the centroids, in a certain order

for dock in docks:
    for name in names:
        for file in glob.glob('/home/jegan/OE_Glide_correlation/csvs/*.csv'):
            for g in range(10):
                if (file.rsplit('/')[5]) == (dock+'_'+name+'_'+str(g)+'.csv'):
                    
                    values = [] #list of the ligands and final values for the heatmap
                    predict_top = [] #list of predicted ligands in top quartile
                    predict_bot = [] #list of predicted ligands in bottom quartile

                    k = open(file,'r')
                    j = 0
                    for h in k:
                        u = h.split(',')
                        j += 1
                        if j <= 115:
                            predict_top.append(u[0])
                        elif j >= 344:
                            predict_bot.append(u[0])

                    ligs = [] #list of ligands that are either correctly put in the top or incorrectly placed outside the bottom
                    for m in ranks_top:
                        for q in predict_top:
                            if m == q:
                                values.append([q,0]) #giving the ligands that are correctly in the top a value of 0
                                ligs.append(q)
                                
                    [values.append([x,1]) and ligs.append(x) for x in ranks_bot if x not in predict_bot]
                    ligs = list(set(ligs))
                    all_ligs = []
                    for o in range(1,460):
                        all_ligs.append('CatS_'+str(o))
                    [values.append([x,2]) for x in all_ligs if x not in ligs]
                    ordered_lig = sorted(values, key = lambda u:float(u[0][5:]))
                    
                    cen_line = []
                    for e in ordered_lig:
                        cen_line.append(e[1])
                        
                    array.append(cen_line)

fig, ax = plt.subplots()
im = ax.imshow(array,cmap = cm.viridis)

cbar = plt.colorbar(im)
cbar.set_label('correlation', rotation = 270, labelpad = 20)
plt.title("Ligands in quartiles per centroid")
#plt.tight_layout()
#plt.savefig(home+'/ligs_per_cen.png')

